# Cornell SoNIC Workshop, Day 1

The research goal this week is to explore covid-19 modeling for a university campus. We will write code to simulate a **S**usceptible-**E**xposed-**I**nfectious-**R**ecovered (SEIR) model and study the sensitivty of this model to various parameters or simplifications. The SEIR model was the basis for Cornell's fall 2020 reactivation plan, which is detailed in the report listed in the readings below. 


Associated readings:
<ul>    
    <li>Networks, Crowds, and Markets: Reasoning about a Highly Connected World, 
        <a href="https://www.cs.cornell.edu/home/kleinber/networks-book/networks-book-ch21.pdf">Chapter 21: Epidemics</a>. David Easley and Jon Kleinberg. </li>
    <li><a href="https://covid.cornell.edu/_assets/files/covid_19_modeling_main_report.pdf">COVID-19 Mathematical Modeling for Cornel's Fall Semester</a> (Sections 1–2.1). J. Massey Cashore, Ning Duan, Alyf Janmohamed, Jiayue Wan, Yujia Zhang, Shane Henderson, David Shmoys, and Peter Frazier.</li>
</ul>

### The SEIR Model

The SEIR model is a [compartmental model](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology). The basic idea is that, at every step, each individual is in one of four states:
<ol>
    <li><i>Susceptible (S)</i>: able to contract the virus</li>
    <li><i>Exposed (E)</i>: already infected but cannot infect others</li>
    <li><i>Infectious (I)</i>: infected and can infect others</li>
    <li><i>Recovered (R)</i>: healthy and no longer able to contract the virus</li>    
</ol>

The virus propagates in steps, and we will think of each step as one day in real time. At each step, susceptible people may become exposed, exposed people may become infectious, and infectious people may infect others and/or recover. We will assue that these events happen roughly "at once" at the end of each day (this is simplification, but a useful one for simulations). There are many ways to model the various compartments of the SEIR model&mdash;the Cornell report is 54 pages! We will examine a few over the next few days.

Let's get started with some code.

In [ ]:
# Libraries we will need today
from enum import Enum, unique
import random
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# The simulation will use these two classes, which does not need editing.

@unique
class SEIR_State(Enum):
    """ Class representing the state of a person. """
    S = 1  # SUSCEPTIBLE
    E = 2  # EXPOSED
    I = 3  # INFECTIOUS
    R = 4  # RECOVERED

class Person:
    """  Class representing a person in the population, keeping track of 
    their state and the number of days (complete simulation steps) that the 
    person has been exposed or infected.
    
    Attributes:
        state: An SEIR_State indicating the state of the person
        days_spent_in_E: an integer number of days as exposed
        days_spent_in_I: an integer number of days as infectious
        max_days_in_E: an integer for the number of days that this person
                       stays in (E) before (I); defaults to 2 for everyone
        max_days_in_I: an integer for the number of days that this person
                       stays in (I) before (R); defaults to 14 for everyone        
    """
    def __init__(self, state):
        self.state = state
        self.days_spent_in_E = 0
        self.days_spent_in_I = 0
        self.max_days_in_E = 2
        self.max_days_in_I = 14

For today, let's make the following assumptions:

<ul>
    <li>Each (I) person initiates contact with a fixed number $c$ of other contacts. These contacts could be in any one of the states (S, E, I, or R), and the $c$ individuals are sampled uniformly at random from the population. For each (S) contact, the infection spreads to that person with probability $p$.</li>
    <li>Every day (each time step), each (S) person also has a probability $r$ of becoming infected from an external source outside of the university population.</li>
    <li>A person is in the (E) state for 2 days (2 complete time steps) before transitioning to (I).</li>
    <li>A person is in the (I) state for 14 days (14 complete time steps) before transitioning to (R).</li>
    <li>At the beginning of the simulation, there are $N_{S,0}$, $N_{E,0}$, $N_{I,0}$, and $N_{R,0}$ people in the S, E, I, and R states. If someone is in the (E) or (I) at initialization, the number of days spent in that state so far is sampled uniformly at random (see below).
</ul>

In [ ]:
# Get a list of people for initializing the simulation. This does not need editing.

def initialize_person_list(N_S0, N_E0, N_I0, N_R0):
    """Form the initial state of the simulation.
    
    Args:
        N_S0: number of susceptible individuals
        N_E0: number of exposed individuals
        N_I0: number of infectious individuals
        N_R0: number of recovered individuals

    Returns:
        List of person objects of length equal 
        to the sum of N_S0, N_E0, N_I0, and N_R0.
    """
    
    plist = []

    for _ in range(N_S0):
        plist.append(Person(SEIR_State.S))

    for _ in range(N_E0):
        person = Person(SEIR_State.E)
        person.days_spent_in_E = np.random.choice(person.max_days_in_E - 1, 1)[0]
        plist.append(person)
        
    for _ in range(N_I0):
        person = Person(SEIR_State.I)
        person.days_spent_in_E = person.max_days_in_E
        person.days_spent_in_I = np.random.choice(person.max_days_in_I - 1, 1)[0]
        plist.append(person)
        
    for _ in range(N_R0):
        person = Person(SEIR_State.R)
        person.days_spent_in_E = person.max_days_in_E
        person.days_spent_in_E = person.max_days_in_I
        plist.append(person)
        
    return plist

In [ ]:
# Main code for a step of the simulation. This needs editing.

def basic_SEIR_step(person_list, p, r, c):
    """Simulate one SEIR step.

    Args:
        person_list: list of person objects
        p: probability of contact transmission
        r: probability of infection from external source 
        c: number of others that one person comes into contact with
    """

    # Determine (S) people that become (E) at the end of this step.
    S_to_E = []
    
    # Determine (E) people that become (I) at the end of this step.
    E_to_I = []

    # Determine (I) people who become (R) at the end of this step.
    I_to_R = []


    # Update S --> E cases
    for index in S_to_E:
        person_list[index].state = SEIR_State.E

    # Update E --> I cases
    for index in E_to_I:
        person_list[index].state = SEIR_State.I

    # Update I --> R cases        
    for index in I_to_R:
        person_list[index].state = SEIR_State.R

In [ ]:
# Basic loop for the simulation. This needs editing.

def run_simulation(N_S0, N_E0, N_I0, N_R0, p, r, c, T):
    """Simulate T SEIR steps.
    
    Args:
        N_S0: number of susceptible individuals at initialization
        N_E0: number of exposed individuals at initialization
        N_I0: number of infectious individuals at initialization
        N_R0: number of recovered individuals at initialization  
        p: probability of contact transmission
        r: probability of infection from external source 
        c: number of others with whom an (I) person initiates contact
        T: number of time steps
    
    Returns:
        data needed for plots (see below)
    """
    
    person_list = initialize_person_list(N_S0, N_E0, N_I0, N_R0)
    
    # Run simulation steps
    for t in range(T):
        basic_SEIR_step(person_list, p, r, c)

Now, let's run the simulation. To do this, we need to set our parameters.

First, determine some reasonable parameter settings. Then, make a plot that shows the number of newly exposed individuals at each time step for $T = 100$ time steps. Make another plot that shows the cumulative fraction of initially susceptible individuals that have become exposed at any point (numpy's <a href="https://numpy.org/doc/stable/reference/generated/numpy.cumsum.html">cumsum function</a> may be useful).

In [ ]:
# Make plots here (need to edit code above)

Next, let's examine the *sensitivity* of our model by seeing how the output responds to certain changes in the input. Make the same plots as before, but overlay curves for varying parameters:
<ol>
    <li>$c$ varies with all other parameters fixed</li>
    <li>$p$ varies with all other parameters fixed</li>
    <li>$r$ varies with all other parameters fixed</li>
</ol>

In [ ]:
# Make plots here

### Basic reproductive number

An epidemiological term you may have heard in the news is $R_0$, pronounced like R-naught or R-zero, which is also sometimes called the basic reproductive number. This is the average number of new infections caused by a single individual who becomes infected. At a high level, if $R_0 < 1$, then the virus will tend to die out and if $R_0 > 1$, the virus will go on to infect most of the population (see the Easley and Kleinberg reading for some of the mathematics). One <a href="https://www.nejm.org/doi/full/10.1056/nejmoa2001316">study</a>
estimated $R_0 \approx 2.2$ in early covid-19 spread.

If $r = 0.0$, we would expect an $R_0$ to be a bit less than $14 \cdot cp$ (why?). Try $14 \cdot cp = 2.0$ and $14 \cdot cp = 0.75$ and see how much of the population becomes exposed.

In [ ]:
# Test of R_0

## Discussion points

<ol>
    <li>What are the biggest simplifications of the SEIR model, and how could we address them?</li>
    <li>How might this model misrepresent certain individuals?</li>
    <li>How should we determine the parameters of the model?</li>
</ol>